In [1]:
import os

In [2]:
%pwd


'c:\\Users\\Siva\\OneDrive\\Desktop\\snekha\\chicken-diseases-classification\\research'

In [3]:
# Changing the directory
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\Siva\\OneDrive\\Desktop\\snekha\\chicken-diseases-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
from pathlib import Path

# Add src/ to sys.path
sys.path.append('src')
sys.path.append(str(Path().resolve() / "src"))





In [29]:
import cnn_Classifier.utils.common as cutils
print(dir(cutils))


['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'create_directories', 'decodeImage', 'download_file_from_google_drive', 'encodeImageIntoBase64', 'ensure_annotations', 'gdown', 'get_confirm_token', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'os', 'read_yaml', 'requests', 'save_bin', 'save_json', 'save_response_content', 'yaml']


In [30]:
import cnn_Classifier.utils.common as cutils
print("Using file:", cutils.__file__)


Using file: c:\users\siva\onedrive\desktop\snekha\chicken-diseases-classification\src\cnn_Classifier\utils\common.py


In [7]:
from cnn_Classifier.constants import *
from cnn_Classifier.utils.common import read_yaml, create_directories


cnn_Classifier __init__.py loaded


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import urllib.request as request
import zipfile
from cnn_Classifier import logger
from cnn_Classifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)






In [11]:
from pathlib import Path
from cnn_Classifier.utils.common import read_yaml, download_file_from_google_drive

CONFIG_FILE_PATH = Path("config/config.yaml")
config = read_yaml(CONFIG_FILE_PATH)

file_id = config.data_ingestion.source_URL
url = f"https://drive.google.com/uc?id={file_id}"
destination = Path(config.data_ingestion.local_data_file)

download_file_from_google_drive(url, destination)


Downloading...
From (original): https://drive.google.com/uc?id=1k2H549BKVQKwcUXpL_K3I8FXEQ5F5Ljm
From (redirected): https://drive.google.com/uc?id=1k2H549BKVQKwcUXpL_K3I8FXEQ5F5Ljm&confirm=t&uuid=65f71b62-fdb9-4485-8693-eb07d08b7732
To: c:\Users\Siva\OneDrive\Desktop\snekha\chicken-diseases-classification\artifacts\data_ingestion\data.zip
100%|██████████| 126M/126M [00:03<00:00, 35.0MB/s] 


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-05 12:35:51,430: INFO: 3797937142: File already exists of size: ~ 123001 KB]
